# XGradient boosting with trees for topic classification
---

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import json

In [2]:
module_path = os.path.abspath(os.path.join('..\..')) # Path to root folder
if module_path not in sys.path:
    sys.path.append(module_path + "/scripts") # define scripts path

from ipynb_func import *

Data loader:

In [3]:
#NUM = 10 # Number of data parquets to use
#assert NUM >= 1 and NUM <= 10, "NUM value must be in range [1, 10]"

# Making list of roots to merge processed raw data 
#paths = [module_path + f"/data/pikabu/tag_processed/raw_data/{i}_tag_processed.parquet" for i in range(NUM)] 

# Making list of roots to merge processed filtered data
#paths = [module_path + f"/data/pikabu/tag_processed/filtered_data/{i}_tag_processed.parquet" for i in range(NUM)] 

# Making list of roots to merge processed cleared data
paths = [module_path + f"/data/pikabu/splited_data/cleared_texts.parquet"] 

data = merge_dataset(paths)

In [4]:
pd.set_option('display.max_colwidth', 130)
data.head(3)

,id,text_markdown,tags
15,6991359,"[добрый, сутки, господин, дама, подсказывать, название, игра, телефон, оформление, убийство, зомби, очки, ездить, машинка, кру...","[игры, поиск]"
37,7004423,"[ехать, девчонка, школа, оставаться, свободный, макс, заявка, прямой, конечный, адрес, железнодорожный, институт, включать, вб...",[юмор]
52,6991603,"[стадо, стадо, гигантский, случаться, стадо, управлять, волк, предел, волк, жопа, враг, дружно, осматривать, выдавливать, стад...",[мат]


---
# 1. Data preparation and split

In [5]:
with open(module_path + f"/data/pikabu/splited_data/indexes.json") as f:
    id_splits = f.read()

id_splits = json.loads(id_splits)

data_train = data[data['id'].isin(id_splits['train'])]
data_val = data[data['id'].isin(id_splits['val'])]
data_test = data[data['id'].isin(id_splits['test'])]

In [6]:
print(f"Number of train data: {len(data_train)}")
print(f"Number of val data: {len(data_val)}")
print(f"Number of test data: {len(data_test)}")
print(f"Distribution: {len(data_train)/len(data)*100:.0f} / {len(data_val)/len(data)*100:.0f} / {len(data_test)/len(data)*100:.0f}")

Number of train data: 25209
Number of val data: 2821
Number of test data: 3119
Distribution: 81 / 9 / 10


---
# 2. Model training

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
import xgboost as xgb
from joblib import dump, load

Target preparation:

In [8]:
Vec = CountVectorizer(tokenizer=lambda x: x.split(','), binary=True)

df = data.copy()
df.tags = [','.join(i) for i in df.tags]

df_train = data_train.copy()
df_train.tags = [','.join(i) for i in df_train.tags]

df_val = data_val.copy()
df_val.tags = [','.join(i) for i in df_val.tags]

df_test = data_test.copy()
df_test.tags = [','.join(i) for i in df_test.tags]

y_data = Vec.fit(df['tags'])
y_train = Vec.transform(df_train['tags'])
y_val = Vec.transform(df_val['tags'])
y_test = Vec.transform(df_test['tags'])

c:\Users\decique\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [9]:
print('Tags to predict:')
print(Vec.get_feature_names_out())

Tags to predict:
['авто' 'авторский рассказ' 'алкоголь' 'анекдот' 'армия' 'вопрос' 'врачи'
 'девушки' 'деньги' 'дети' 'детство' 'другое' 'жизнь' 'игры' 'интересное'
 'истории' 'история' 'ищу книгу' 'ищу фильм' 'карантин' 'книги'
 'коронавирус' 'кот' 'лига добра' 'лига юристов' 'любовь' 'люди' 'мат'
 'медицина' 'москва' 'музыка' 'мысли' 'негатив' 'новости' 'новый год'
 'общество' 'отношения' 'поиск' 'политика' 'помогите найти' 'помощь'
 'психология' 'работа' 'рассказ' 'реальная история из жизни'
 'родители и дети' 'россия' 'самоизоляция' 'санкт-петербург' 'семья'
 'случай из жизни' 'совет' 'сон' 'соседи' 'стихи' 'украина' 'фантастика'
 'фильмы' 'школа' 'юмор']


In [10]:
tag_distr = getworddict(getwordlist(data.tags))
tag_distr_formated = {}
for i in range(len(tag_distr)):
    tag_distr_formated[i] = round(tag_distr[Vec.get_feature_names_out()[i]] / sum(tag_distr.values()), 4)

In [11]:
print('Tags weights:')
print(tag_distr_formated)

Tags weights:
{0: 0.0073, 1: 0.0121, 2: 0.0075, 3: 0.0079, 4: 0.0085, 5: 0.0196, 6: 0.0077, 7: 0.0138, 8: 0.0089, 9: 0.0279, 10: 0.011, 11: 0.0221, 12: 0.0186, 13: 0.0122, 14: 0.0122, 15: 0.0086, 16: 0.0338, 17: 0.0067, 18: 0.008, 19: 0.014, 20: 0.0107, 21: 0.0454, 22: 0.0121, 23: 0.0142, 24: 0.0078, 25: 0.0149, 26: 0.0084, 27: 0.0549, 28: 0.0125, 29: 0.0098, 30: 0.0081, 31: 0.0093, 32: 0.0145, 33: 0.0152, 34: 0.0133, 35: 0.0237, 36: 0.0206, 37: 0.0071, 38: 0.0308, 39: 0.0118, 40: 0.054, 41: 0.0121, 42: 0.0297, 43: 0.0303, 44: 0.0315, 45: 0.0082, 46: 0.0277, 47: 0.0063, 48: 0.0075, 49: 0.0145, 50: 0.0122, 51: 0.0078, 52: 0.0067, 53: 0.0086, 54: 0.0274, 55: 0.0275, 56: 0.0111, 57: 0.0104, 58: 0.0144, 59: 0.0384}


In [12]:
print('Y shapes:')
print(f'  • Y train: {y_train.shape}')
print(f'  • Y validation: {y_val.shape}')
print(f'  • Y test: {y_test.shape}')

Y shapes:
  • Y train: (25209, 60)
  • Y validation: (2821, 60)
  • Y test: (3119, 60)


---

## 2.1. Training with bag-of-words embeddings:

In [13]:
save_models_path = module_path + '/models/xgb/'

In [14]:
X_data = [' '.join(txt) for txt in data.text_markdown]

In [15]:
X_train = [' '.join(txt) for txt in data_train.text_markdown]
X_val = [' '.join(txt) for txt in data_val.text_markdown]
X_test = [' '.join(txt) for txt in data_test.text_markdown]

X_Vec = CountVectorizer(tokenizer = lambda x: x.split())

X_Vec.fit(X_train)
X_train = X_Vec.transform(X_train)
X_test = X_Vec.transform(X_test)
X_val = X_Vec.transform(X_val)

In [16]:
print("X BoW's shapes:")
print(f'   • X train shape: {X_train.shape}')
print(f'   • X val shape: {X_val.shape}')
print(f'   • X test shape: {X_test.shape}')

X BoW's shapes:
   • X train shape: (25209, 5899)
   • X val shape: (2821, 5899)
   • X test shape: (3119, 5899)


In [17]:
""" # Searching for best model params; too long;

from sklearn.model_selection import GridSearchCV

cfg = {'estimator__n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
       'estimator__max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
       'estimator__learning_rate': [1, 10, 100, 1000, 1e5]}

clf_ovr = OneVsRestClassifier(estimator=xgb.XGBClassifier(n_estimators=5,
                                                          max_depth=5,
                                                          learning_rate=10,
                                                          objective='binary:logistic',
                                                          random_state=42),
                              n_jobs=-1)

GSCV_clf = GridSearchCV(estimator=clf_ovr, param_grid=cfg, n_jobs=-1)

GSCV_clf.fit(X_train, y_train)

GSCV_params = GSCV_clf.best_params_  """;

In [18]:
clf_ovr = OneVsRestClassifier(estimator=xgb.XGBClassifier(tree_method='auto',
                                                          n_estimators=1,
                                                          max_depth=25,
                                                          learning_rate=1e5,
                                                          objective='binary:logistic',
                                                          random_state=42),
                              n_jobs=-1)

In [19]:
if os.path.isfile(save_models_path + 'bow.joblib'):
    clf_ovr = load(save_models_path + 'bow.joblib')
else:
    clf_ovr.fit(X_train, y_train)
    dump(clf_ovr, save_models_path + 'bow.joblib')

In [20]:
bow_y_pred_val = clf_ovr.predict(X_val)
bow_y_probas = clf_ovr.predict_proba(X_val)
bow_y_labels = sorted_labels(bow_y_pred_val, bow_y_probas, Vec)

bow_df_val = data_val.copy()
bow_df_val['predicted_tags'] = bow_y_labels

In [21]:
bow_df_val.head(5)

,id,text_markdown,tags,predicted_tags
421,6992880,"[популярный, пк, игра, создавать, устройство, проходить, картинка, память, оставаться, различный, создавать, написать, догадка...",[помогите найти],[интересное]
432,6992917,"[профессия, оказываться, сопровождать, образование, курсы, обязанность, входить, хотеться, разговор, зажигать, свеча, оказыват...",[психология],[]
578,6994231,"[предыдущий, пост, бригада, график, переписывать, месяц, дата, окончание, поездка, графика, случаться, называть, двойной, поез...","[юмор, реальная история из жизни]","[анекдот, любовь, работа]"
591,6992488,"[широкий, хотеться, обращать, внимание, инцидент, связанный, дтп, участие, пожарный, двигаться, оперативный, вызов, включать, ...",[помощь],"[лига юристов, помощь]"
807,7021892,"[история, скоро, девушка, упасть, дерево, парень, неловко, улыбаться, следовать, забывать, следить, окружение, темно, отзывать...","[рассказ, фантастика, мат]","[авторский рассказ, детство, игры, истории, любовь, люди, мысли, поиск, помощь, рассказ, родители и дети, фантастика, юмор]"


In [22]:
print('Metrics for Bag-of-Words:')
print(classification_report(y_val, bow_y_pred_val, zero_division=0))

Metrics for Bag-of-Words:
              precision    recall  f1-score   support

           0       0.16      0.52      0.25        25
           1       0.15      0.36      0.21        45
           2       0.20      0.38      0.26        40
           3       0.09      0.15      0.11        27
           4       0.19      0.56      0.28        27
           5       0.05      0.12      0.07        68
           6       0.23      0.38      0.29        40
           7       0.16      0.41      0.23        59
           8       0.10      0.19      0.13        42
           9       0.18      0.46      0.26       113
          10       0.11      0.26      0.15        50
          11       0.02      0.01      0.01        85
          12       0.07      0.10      0.08        72
          13       0.40      0.81      0.54        58
          14       0.05      0.06      0.06        52
          15       0.01      0.04      0.02        28
          16       0.09      0.11      0.10       135
 

Calculate `recall@k`:

In [23]:
bow_recallk = recallk(bow_df_val.tags, bow_df_val.predicted_tags, k=10)
print(f'Recall@k for Bag-of-Words: {bow_recallk:.4f}')

Recall@k for Bag-of-Words: 0.5487


## 2.2 Training with IF-IDF embeddings:

In [24]:
X_train = [' '.join(txt) for txt in data_train.text_markdown]
X_val = [' '.join(txt) for txt in data_val.text_markdown]
X_test = [' '.join(txt) for txt in data_test.text_markdown]

Tfidf_Vec = TfidfVectorizer(tokenizer = lambda x: x.split())

Tfidf_Vec.fit(X_train)
X_train = Tfidf_Vec.transform(X_train)
X_test = Tfidf_Vec.transform(X_test)
X_val = Tfidf_Vec.transform(X_val)

c:\Users\decique\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [25]:
print("X TF-IDF's shapes:")
print(f'   • X train shape: {X_train.shape}')
print(f'   • X val shape: {X_val.shape}')
print(f'   • X test shape: {X_test.shape}')

X TF-IDF's shapes:
   • X train shape: (25209, 5899)
   • X val shape: (2821, 5899)
   • X test shape: (3119, 5899)


In [26]:
clf_ovr = OneVsRestClassifier(estimator=xgb.XGBClassifier(tree_method='auto',
                                                          n_estimators=5,
                                                          max_depth=15,
                                                          learning_rate=1e5,
                                                          objective='binary:logistic',
                                                          random_state=42),
                              n_jobs=-1)

In [27]:
if os.path.isfile(save_models_path + 'tf_idf.joblib'):
    clf_ovr = load(save_models_path + 'tf_idf.joblib')
else:
    clf_ovr.fit(X_train, y_train)
    dump(clf_ovr, save_models_path + 'tf_idf.joblib')

In [28]:
tf_idf_y_pred_val = clf_ovr.predict(X_val)
tf_idf_y_probas = clf_ovr.predict_proba(X_val)
tf_idf_y_labels = sorted_labels(tf_idf_y_pred_val, tf_idf_y_probas, Vec)

tf_idf_df_val = data_val.copy()
tf_idf_df_val['predicted_tags'] = tf_idf_y_labels

In [29]:
tf_idf_df_val.head(5)

,id,text_markdown,tags,predicted_tags
421,6992880,"[популярный, пк, игра, создавать, устройство, проходить, картинка, память, оставаться, различный, создавать, написать, догадка...",[помогите найти],[игры]
432,6992917,"[профессия, оказываться, сопровождать, образование, курсы, обязанность, входить, хотеться, разговор, зажигать, свеча, оказыват...",[психология],"[психология, работа]"
578,6994231,"[предыдущий, пост, бригада, график, переписывать, месяц, дата, окончание, поездка, графика, случаться, называть, двойной, поез...","[юмор, реальная история из жизни]",[]
591,6992488,"[широкий, хотеться, обращать, внимание, инцидент, связанный, дтп, участие, пожарный, двигаться, оперативный, вызов, включать, ...",[помощь],"[вопрос, лига юристов]"
807,7021892,"[история, скоро, девушка, упасть, дерево, парень, неловко, улыбаться, следовать, забывать, следить, окружение, темно, отзывать...","[рассказ, фантастика, мат]","[авторский рассказ, книги, рассказ]"


In [30]:
print('Metrics for TF-IDF:')
print(classification_report(y_val, tf_idf_y_pred_val, zero_division=0))

Metrics for TF-IDF:
              precision    recall  f1-score   support

           0       0.16      0.40      0.23        25
           1       0.20      0.36      0.26        45
           2       0.25      0.40      0.31        40
           3       0.20      0.22      0.21        27
           4       0.29      0.56      0.38        27
           5       0.12      0.18      0.15        68
           6       0.27      0.42      0.33        40
           7       0.18      0.36      0.24        59
           8       0.12      0.17      0.14        42
           9       0.25      0.37      0.30       113
          10       0.13      0.22      0.16        50
          11       0.06      0.04      0.05        85
          12       0.09      0.07      0.08        72
          13       0.42      0.78      0.54        58
          14       0.03      0.02      0.02        52
          15       0.02      0.04      0.03        28
          16       0.11      0.09      0.10       135
       

Calculate `recall@k`:

In [31]:
tf_idf_recallk = recallk(tf_idf_df_val.tags, tf_idf_df_val.predicted_tags, k=10)
print(f'Recall@k on TF-IDF: {tf_idf_recallk:.4f}')

Recall@k on TF-IDF: 0.5186


## 2.3. Training on TF-IDF with N-grams:

In [32]:
X_train = [' '.join(txt) for txt in data_train.text_markdown]
X_val = [' '.join(txt) for txt in data_val.text_markdown]
X_test = [' '.join(txt) for txt in data_test.text_markdown]

Tfidf_Vec = TfidfVectorizer(tokenizer = lambda x: x.split(), ngram_range=(1, 2))

Tfidf_Vec.fit(X_train)
X_train = Tfidf_Vec.transform(X_train)
X_test = Tfidf_Vec.transform(X_test)
X_val = Tfidf_Vec.transform(X_val)

c:\Users\decique\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [33]:
print("X TF-IDF with n-grams's shapes:")
print(f'   • X train shape: {X_train.shape}')
print(f'   • X val shape: {X_val.shape}')
print(f'   • X test shape: {X_test.shape}')

X TF-IDF with n-grams's shapes:
   • X train shape: (25209, 1654803)
   • X val shape: (2821, 1654803)
   • X test shape: (3119, 1654803)


In [34]:
clf_ovr = OneVsRestClassifier(estimator=xgb.XGBClassifier(tree_method='auto',
                                                          n_estimators=3,
                                                          max_depth=13,
                                                          learning_rate=1e5,
                                                          objective='binary:logistic',
                                                          random_state=42),
                              n_jobs=1)

In [35]:
if os.path.isfile(save_models_path + 'tf_idf_ngrams.joblib'):
    clf_ovr = load(save_models_path + 'tf_idf_ngrams.joblib')
else:
    clf_ovr.fit(X_train, y_train)
    dump(clf_ovr, save_models_path + 'tf_idf_ngrams.joblib')

In [36]:
n_gram_tf_idf_y_pred_val = clf_ovr.predict(X_val)
n_gram_tf_idf_y_probas = clf_ovr.predict_proba(X_val)
n_gram_tf_idf_y_labels = sorted_labels(n_gram_tf_idf_y_pred_val, n_gram_tf_idf_y_probas, Vec)

n_gram_tf_idf_df_val = data_val.copy()
n_gram_tf_idf_df_val['predicted_tags'] = n_gram_tf_idf_y_labels

In [37]:
n_gram_tf_idf_df_val.head(5)

,id,text_markdown,tags,predicted_tags
421,6992880,"[популярный, пк, игра, создавать, устройство, проходить, картинка, память, оставаться, различный, создавать, написать, догадка...",[помогите найти],[игры]
432,6992917,"[профессия, оказываться, сопровождать, образование, курсы, обязанность, входить, хотеться, разговор, зажигать, свеча, оказыват...",[психология],"[ищу книгу, психология, работа]"
578,6994231,"[предыдущий, пост, бригада, график, переписывать, месяц, дата, окончание, поездка, графика, случаться, называть, двойной, поез...","[юмор, реальная история из жизни]","[любовь, работа]"
591,6992488,"[широкий, хотеться, обращать, внимание, инцидент, связанный, дтп, участие, пожарный, двигаться, оперативный, вызов, включать, ...",[помощь],"[лига юристов, помощь]"
807,7021892,"[история, скоро, девушка, упасть, дерево, парень, неловко, улыбаться, следовать, забывать, следить, окружение, темно, отзывать...","[рассказ, фантастика, мат]","[авторский рассказ, девушки, рассказ, реальная история из жизни, фантастика]"


In [38]:
print('Metrics for TF-IDF with 1-2 n-grams:')
print(classification_report(y_val, n_gram_tf_idf_y_pred_val, zero_division=0))

Metrics for TF-IDF with 1-2 n-grams:
              precision    recall  f1-score   support

           0       0.22      0.60      0.32        25
           1       0.23      0.36      0.28        45
           2       0.21      0.50      0.30        40
           3       0.15      0.30      0.20        27
           4       0.24      0.63      0.35        27
           5       0.12      0.18      0.14        68
           6       0.17      0.40      0.24        40
           7       0.18      0.56      0.27        59
           8       0.13      0.24      0.17        42
           9       0.25      0.47      0.33       113
          10       0.15      0.30      0.20        50
          11       0.04      0.04      0.04        85
          12       0.12      0.10      0.11        72
          13       0.37      0.90      0.52        58
          14       0.00      0.00      0.00        52
          15       0.00      0.00      0.00        28
          16       0.12      0.10      0.11 

Calculate `recall@k`:

In [39]:
n_gram_tf_idf_recallk = recallk(n_gram_tf_idf_df_val.tags, n_gram_tf_idf_df_val.predicted_tags, k=10)
print(f'Recall@k for TF-IDF with 1-2 n-grams: {n_gram_tf_idf_recallk:.4f}')

Recall@k for TF-IDF with 1-2 n-grams: 0.5874


## 2.4. Training on rubert-tiny-v2 embeddings:

In [40]:
emb_paths = module_path + '/data/embeddings/rubert-tiny-v2/'

emb_pth = [emb_paths + 'texts.parquet']
emb = merge_dataset(emb_pth)

In [41]:
emb.head(3)

,id,embedding
0,2936217,"[0.3411005, -0.16877297, -0.3599054, 0.011505239, -0.19693527, 0.16206133, -0.62560713, -0.38459125, -0.08364315, -0.17384137,..."
1,6991412,"[0.3696494, 0.06409113, -0.62138826, -0.8906186, 0.08984075, 0.27482352, -0.31647494, -0.778525, -0.6068895, 0.42193377, -0.05..."
2,6991359,"[0.3278318, 0.08586374, -0.7452521, -0.2529353, -0.32851255, 0.5415312, -0.53284395, -1.2018805, 0.120118916, 0.15034527, -0.3..."


In [42]:
emb = emb[emb['id'].isin(data['id'])]

emb_train = emb[emb['id'].isin(data_train['id'])]
emb_val = emb[emb['id'].isin(data_val['id'])]
emb_test = emb[emb['id'].isin(data_test['id'])]

assert len(emb_train) == len(data_train), "Something went wrong!"
assert len(emb_val) == len(data_val), "Something went wrong!"
assert len(emb_test) == len(data_test), "Something went wrong!"

In [43]:
X_train_emb = [i for i in emb_train.embedding]
X_val_emb = [i for i in emb_val.embedding]
X_test_emb = [i for i in emb_test.embedding]

In [44]:
print("X embeddings shapes:")
print(f'   • X train shape: {np.shape(X_train_emb)}')
print(f'   • X val shape: {np.shape(X_val_emb)}')
print(f'   • X test shape: {np.shape(X_test_emb)}')

X embeddings shapes:
   • X train shape: (25209, 312)
   • X val shape: (2821, 312)
   • X test shape: (3119, 312)


In [45]:
clf_ovr = OneVsRestClassifier(estimator=xgb.XGBClassifier(tree_method='auto',
                                                          n_estimators=5,
                                                          max_depth=20,
                                                          learning_rate=1e5,
                                                          objective='binary:logistic',
                                                          random_state=42),
                              n_jobs=1)

In [46]:
if os.path.isfile(save_models_path + 'rubert.joblib'):
    clf_ovr = load(save_models_path + 'rubert.joblib')
else:
    clf_ovr.fit(X_train_emb, y_train)
    dump(clf_ovr, save_models_path + 'rubert.joblib')

In [47]:
rubert_y_pred_val = clf_ovr.predict(X_val_emb)
rubert_y_probas = clf_ovr.predict_proba(X_val_emb)
rubert_y_labels = sorted_labels(rubert_y_pred_val, rubert_y_probas, Vec)

rubert_df_val = data_val.copy()
rubert_df_val['predicted_tags'] = rubert_y_labels

In [48]:
rubert_df_val.head(5)

,id,text_markdown,tags,predicted_tags
421,6992880,"[популярный, пк, игра, создавать, устройство, проходить, картинка, память, оставаться, различный, создавать, написать, догадка...",[помогите найти],"[игры, помогите найти]"
432,6992917,"[профессия, оказываться, сопровождать, образование, курсы, обязанность, входить, хотеться, разговор, зажигать, свеча, оказыват...",[психология],[вопрос]
578,6994231,"[предыдущий, пост, бригада, график, переписывать, месяц, дата, окончание, поездка, графика, случаться, называть, двойной, поез...","[юмор, реальная история из жизни]","[жизнь, история, мат, работа, реальная история из жизни]"
591,6992488,"[широкий, хотеться, обращать, внимание, инцидент, связанный, дтп, участие, пожарный, двигаться, оперативный, вызов, включать, ...",[помощь],"[армия, новости, помогите найти, помощь]"
807,7021892,"[история, скоро, девушка, упасть, дерево, парень, неловко, улыбаться, следовать, забывать, следить, окружение, темно, отзывать...","[рассказ, фантастика, мат]","[авторский рассказ, история, книги, мат, рассказ, реальная история из жизни, фантастика]"


In [49]:
print('Metrics for rubert embeddings:')
print(classification_report(y_val, rubert_y_pred_val, zero_division=0))

Metrics for rubert embeddings:
              precision    recall  f1-score   support

           0       0.06      0.16      0.09        25
           1       0.17      0.33      0.22        45
           2       0.15      0.23      0.18        40
           3       0.03      0.07      0.04        27
           4       0.08      0.19      0.11        27
           5       0.08      0.19      0.11        68
           6       0.10      0.12      0.11        40
           7       0.10      0.22      0.14        59
           8       0.11      0.17      0.13        42
           9       0.10      0.23      0.14       113
          10       0.10      0.22      0.13        50
          11       0.05      0.11      0.07        85
          12       0.05      0.11      0.07        72
          13       0.21      0.40      0.27        58
          14       0.04      0.12      0.06        52
          15       0.01      0.04      0.02        28
          16       0.09      0.20      0.12       

Calculate `recall@k`:

In [50]:
rubert_recallk = recallk(rubert_df_val.tags, rubert_df_val.predicted_tags, k=10)
print(f'Recall@k for model with rubert embeddings: {rubert_recallk:.4f}')

Recall@k for model with rubert embeddings: 0.3701


---
# 3. Results

As the result of training with different embeddings, we have the following:

In [51]:
print("Recall@k's with k=10 for models:\n")
print(f'Recall@k for Bag-of-Words: {bow_recallk:3f}')
print(f'Recall@k for TF-IDF: {tf_idf_recallk:3f}')
print(f'Recall@k for TF-IDF with uni- and bi- grams: {n_gram_tf_idf_recallk:3f}')
print(f'Recall@k for rubert embeddings: {rubert_recallk:3f}')

Recall@k's with k=10 for models:

Recall@k for Bag-of-Words: 0.548742
Recall@k for TF-IDF: 0.518610
Recall@k for TF-IDF with uni- and bi- grams: 0.587380
Recall@k for rubert embeddings: 0.370082
